In [1]:
import transformers
import torch
import tensorflow
import pandas as pd
import string
from collections import defaultdict
from tqdm import tqdm

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM

from transformers.generation.configuration_utils import GenerationConfig
from sentence_transformers import SentenceTransformer
import re
import numpy as np
from random import random
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

2024-07-29 19:13:40.075696: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-29 19:13:40.144301: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-29 19:13:41.380045: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
import pandas as pd
dfs = pd.read_csv("pic.csv")
dfs.shape

(2000, 6)

In [14]:
df_label_1 = dfs[dfs['label'] == 1]
df_label_0 = dfs[dfs['label'] == 0]

# Randomly sample 250 rows from each
df_label_1_sample = df_label_1.sample(n=250, random_state=1)
df_label_0_sample = df_label_0.sample(n=250, random_state=1)

# Concatenate the two samples to form the new dataframe
df = pd.concat([df_label_1_sample, df_label_0_sample])

# Shuffle the rows to mix labels
df = df.sample(frac=1, random_state=1).reset_index(drop=True)
df.shape

(500, 6)

In [15]:
df['input'] = 'Phrase1- ' + df['phrase1'] + '\nPhrase2- ' + df['phrase2']

In [16]:
model_path = "/projects/rtelidevara/shared-lib/rtelidevara/llama3-instruct/"  

tokenizer = AutoTokenizer.from_pretrained(model_path)
generation_config = GenerationConfig.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
zero_shot_system_prompt = """
You are an advanced language model designed to analyze pairs of sentences to determine if they are paraphrases of each other. Your task is to give a score of 0 or 1 based on the criteria provided below:
Input format:
Phrase1-..
Phrase2-..

Scoring Criteria:
Score of 1: The two phrases convey the same meaning or very similar meanings. Despite potential differences in wording, structure, or lexical choices, both phrases should represent the same underlying message or information.
Minor differences in phrasing, synonyms, or sentence construction should not affect the overall similarity in meaning.

Score of 0:The two phrases do not convey the same meaning. Even if the phrases share many words or have high lexical overlap, they should be scored as 0 if they represent different meanings or provide different information.
Any significant difference in the message, intent, or factual content between the phrases should result in a score of 0.

Following these criterion, analyse the phrases and output just the score, don't output any reasoning.
"""

In [19]:
def llama_v3_prompt(messages, system_prompt):
    B_INST, E_INST = "<|start_header_id|>{user}<|end_header_id|>", "<|eot_id|>"
    B_SYS, E_SYS = "<|start_header_id|>{system}<|end_header_id|>", "<|eot_id|>"
    BOS, EOS = "<|begin_of_text|>", "<|end_of_text|>"
    ASSISTANT = "<|start_header_id|>assistant<|end_header_id|>"
    DEFAULT_SYSTEM_PROMPT = system_prompt

    if messages[0]["role"] == "system":
        first = messages[0]
        messages = messages[1:]
    else:
        first = {
                "role": "system",
                "content": DEFAULT_SYSTEM_PROMPT.strip(),
            }
    first_list = [BOS + B_SYS + "\n\n" + first["content"].strip() + E_SYS]
    messages_list = []
    count = 0
    for prompt, answer in zip(messages[::2], messages[1::2]):
        if count != 0:
          messages_list.append(f"{BOS}{B_INST}\n\n{(prompt['content']).strip()}{E_INST}{ASSISTANT}\n\n{(answer['content']).strip()}{E_INST}{EOS}")
        else:
          messages_list.append(f"{B_INST}\n\n{(prompt['content']).strip()}{E_INST}{ASSISTANT}\n\n{(answer['content']).strip()}{E_INST}{EOS}")
        count += 1

    if len(messages) != 1:
      messages_list.append(f"{BOS}{B_INST}\n\n{(messages[-1]['content']).strip()}{E_INST}{ASSISTANT}")
    else:
      messages_list.append(f"{B_INST}\n\n{(messages[-1]['content']).strip()}{E_INST}{ASSISTANT}")

    return "".join(first_list + messages_list)

In [20]:
def get_prompt(dialogs, system_prompt):
    temp = llama_v3_prompt(dialogs, system_prompt)
    prompt = f"""{temp}"""
    return prompt

In [21]:
def get_response(prompt):
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=800)
    result = pipe(f"{prompt}")
    generated_text = result[0]['generated_text']

    if generated_text.startswith(prompt):
        final_response = generated_text[len(prompt):]
    else:
        final_response = generated_text 

    final_response = final_response.strip()
    return final_response

In [22]:
predicted_labels = []

# Iterate over each row in the DataFrame and get the response from the LLM
for index, row in df.iterrows():
    text = row['input']
    
    dialogs = [{"role": "user",
                "content": text}]
    prompt = get_prompt(dialogs, zero_shot_system_prompt)
    response = get_response(prompt)
    # print(text)
    # print("--")
    # print(response)
    predicted_labels.append(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [23]:
def convert_to_int(s):
    if 'Score: ' in s:
        return int(s.split(': ')[1])
    else:
        return int(s)

In [24]:
predicted_labels = [convert_to_int(x) for x in predicted_labels if x in ['0', '1', 'Score: 0', 'Score: 1']]

zero_shot_results_df = df.copy()
zero_shot_results_df['predicted_label'] = predicted_labels

In [25]:
print(predicted_labels)

[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [26]:
mismatch_count = (zero_shot_results_df['label'] != zero_shot_results_df['predicted_label']).sum()
print(mismatch_count)

255


In [28]:
few_shot_system_prompt = """
You are an advanced language model designed to analyze pairs of sentences to determine if they are paraphrases of each other. Your task is to give a score of 0 or 1 based on the criteria provided below:
Input format:
phrase1-..
phrase2-...

Scoring Criteria:
Score of 1: The two sentences convey the same meaning or very similar meanings. Despite potential differences in wording, structure, or lexical choices, both sentences should represent the same underlying message or information.
Minor differences in phrasing, synonyms, or sentence construction should not affect the overall similarity in meaning.

Score of 0:The two sentences do not convey the same meaning. Even if the sentences share many words or have high lexical overlap, they should be scored as 0 if they represent different meanings or provide different information.
Any significant difference in the message, intent, or factual content between the sentences should result in a score of 0.

Here are some examples:
1. phrase1- newly formed camp
phrase2- recently made encampment	
Score: 0
    
2. phrase1- involved people	
phrase2- participating individuals	
Score: 1

3. phrase1- major criticism	
phrase2- Harsh outspoken discussion	
Score: 0

4. phrase1- good prospects	
phrase2- great chance	
Score: 1

5. phrase1- new practice	
phrase2- modern application	
Score: 0

6. phrase1- program's design	
phrase2- project's layout
Score: 1
    
Understand the patterns from the examples, and following these criterion, analyse the phrases and output just the score, don't output any reasoning.
"""

In [29]:
predicted_labels = []

# Iterate over each row in the DataFrame and get the response from the LLM
for index, row in df.iterrows():
    text = row['input']
    
    dialogs = [{"role": "user",
                "content": text}]
    prompt = get_prompt(dialogs, few_shot_system_prompt)
    response = get_response(prompt)
    # print(text)
    # print("--")
    # print(response)
    predicted_labels.append(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [30]:
print(predicted_labels)

['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1',

In [31]:
predicted_labels = [convert_to_int(x) for x in predicted_labels if x in ['0', '1', 'Score: 0', 'Score: 1']]

few_shot_results_df = df.copy()
few_shot_results_df['predicted_label'] = predicted_labels

In [32]:
mismatch_count = (few_shot_results_df['label'] != few_shot_results_df['predicted_label']).sum()
print(mismatch_count)

247
